# Q3

In [6]:
import numpy as np
import pandas as pd

dataset = [
    {"Age": 25, "Income": "High", "Student": "No", "Credit": "Fair", "Buy": "No"},
    {"Age": 30, "Income": "High", "Student": "No",  "Credit": "Excellent", "Buy": "No"},
    {"Age": 35, "Income": "Medium", "Student": "No",  "Credit": "Fair", "Buy": "Yes"},
    {"Age": 40, "Income": "Low", "Student": "Yes", "Credit": "Fair", "Buy": "Yes"},
    {"Age": 45, "Income": "Low", "Student": "Yes", "Credit": "Excellent", "Buy": "Yes"},
    {"Age": 50, "Income": "Low", "Student": "No",  "Credit": "Excellent", "Buy": "No"},
    {"Age": 55, "Income": "Medium", "Student": "No",  "Credit": "Excellent", "Buy": "Yes"},
    {"Age": 60, "Income": "High", "Student": "Yes", "Credit": "Fair", "Buy": "No"}
]

df = pd.DataFrame(dataset)
print(df)                                             # just to verify the data

print(df.columns)

   Age  Income Student     Credit  Buy
0   25    High      No       Fair   No
1   30    High      No  Excellent   No
2   35  Medium      No       Fair  Yes
3   40     Low     Yes       Fair  Yes
4   45     Low     Yes  Excellent  Yes
5   50     Low      No  Excellent   No
6   55  Medium      No  Excellent  Yes
7   60    High     Yes       Fair   No
Index(['Age', 'Income', 'Student', 'Credit', 'Buy'], dtype='object')


## Gini Impurity
As in class, the loss function Gini Index/Impurity is defined by:
<br>
G = Σ P mk(1 - P mk), summed over all classes k. This is the same as:
<br>
G = Σ P mk - Σ (P mk)^2 = 1 - Σ (P mk)^2.
<br>
I use this formula to compute the G.

In [3]:
def gini_impurity(labels):
    """
    y: array-like of labels (e.g., 'Yes' or 'No')
    Returns the Gini impurity of this set of labels.
    """
    unique_labels, counts = np.unique(labels, return_counts = True)
    probabilities = counts / counts.sum()
    g = 1 - np.sum(probabilities**2)                     # Gini = 1 - sum(prob^2)
    return g

## Finding the Best Split

In [29]:
def is_numeric(datatype):
    """ Determines if the feature is numeric(like age) or categorical(all the others). """
    return pd.api.types.is_numeric_dtype(datatype)

def find_best_split(X, y):
    """
    Inputs -
    X: DataFrame of features, y: Series of labels
    Returns: Details of best split
    """
    best_feature = None
    best_value = None
    best_gini_split = float('inf')            # we minimize a loss function, so initially it is set to max
    best_left_index = None
    best_right_index = None

    n_samples = len(y)

    for feature in X.columns:                               # printing the feature will just print its name, ie., the columnname of the dataframe
        unique_vals = X[feature].unique()

        # Numeric feature
        if is_numeric(X[feature].dtype):
            sorted_vals = np.sort(unique_vals)

            # Computing the gini for each unique value of the feature
            for val in sorted_vals:
                left_index = X[feature] <= val
                right_index = X[feature] > val
                if left_index.sum() == 0 or right_index.sum() == 0:
                    continue

                # Compute weighted Gini
                left_gini = gini_impurity(y[left_index])
                right_gini = gini_impurity(y[right_index])
                w_left = left_index.sum() / n_samples
                w_right = right_index.sum() / n_samples
                gini_split = w_left * left_gini + w_right * right_gini

                # Update best split
                if gini_split < best_gini_split:
                    best_gini_split = gini_split
                    best_feature = feature
                    best_value = val
                    best_left_index = left_index
                    best_right_index = right_index
        else:
            # Categorical feauter
            for val in unique_vals:
                left_index = X[feature] == val
                right_index = X[feature] != val
                if left_index.sum() == 0 or right_index.sum() == 0:
                    continue

                # Compute weighted Gini
                left_gini = gini_impurity(y[left_index])
                right_gini = gini_impurity(y[right_index])
                w_left = left_index.sum() / n_samples
                w_right = right_index.sum() / n_samples
                gini_split = w_left * left_gini + w_right * right_gini

                # Update best split
                if gini_split < best_gini_split:
                    best_gini_split = gini_split
                    best_feature = feature
                    best_value = val
                    best_left_index = left_index
                    best_right_index = right_index

    return best_feature, best_value, best_gini_split, best_left_index, best_right_index

## Building the Tree

In [32]:
def majority_class(y):
    """Returns the most common label in y."""
    return y.value_counts().idxmax()

def build_tree(X, y, depth=0, max_depth=5, min_samples_split=2):
    if len(np.unique(y)) == 1:
        return {"leaf": True, "prediction": y.iloc[0]}

    if depth >= max_depth or len(y) < min_samples_split:
        return {"leaf": True, "prediction": majority_class(y)}

    # compute the best split
    best_feature, best_value, best_gini_split, left_index, right_index = find_best_split(X, y)

    # If there was no valid split, just return a leaf
    if best_feature is None:
        return {"leaf": True, "prediction": majority_class(y)}

    node = {
        "leaf": False,
        "feature": best_feature,
        "value": best_value,
        "gini": best_gini_split
    }

    # Build subtrees
    X_left, y_left = X[left_index], y[left_index]
    X_right, y_right = X[right_index], y[right_index]

    node["left"] = build_tree(X_left, y_left, depth + 1, max_depth, min_samples_split)
    node["right"] = build_tree(X_right, y_right, depth + 1, max_depth, min_samples_split)

    return node


## Method for Predicting new Test Cases

In [31]:
def predict_sample(sample, tree):
    """
    sample: a single row of features. can be a dict, or pd.series.
    tree: dictionary for the decision tree
    """
    # If it's a leaf node, return the stored prediction
    if tree["leaf"]:
        return tree["prediction"]

    feature = tree["feature"]
    value = tree["value"]

    if is_numeric(value):
        # Numeric split
        if sample[feature] <= value:
            return predict_sample(sample, tree["left"])
        else:
            return predict_sample(sample, tree["right"])
    else:
        # Categorical split
        if sample[feature] == value:
            return predict_sample(sample, tree["left"])
        else:
            return predict_sample(sample, tree["right"])


## Training the Decision Tree

In [20]:
# Main
import pprint

# Buy is our Label. Classes: Yes and No
X = df.drop("Buy", axis=1)
y = df["Buy"]

# Build the tree
tree = build_tree(X, y, max_depth = 5, min_samples_split = 2)

pprint.pprint(tree)

{'feature': 'Income',
 'gini': np.float64(0.1999999999999999),
 'leaf': False,
 'left': {'leaf': True, 'prediction': 'No'},
 'right': {'feature': 'Age',
           'gini': np.float64(0.2),
           'leaf': False,
           'left': {'leaf': True, 'prediction': 'Yes'},
           'right': {'feature': 'Age',
                     'gini': np.float64(0.0),
                     'leaf': False,
                     'left': {'leaf': True, 'prediction': 'No'},
                     'right': {'leaf': True, 'prediction': 'Yes'},
                     'value': np.int64(50)},
           'value': np.int64(45)},
 'value': 'High'}


## Classifying the Test Case

In [27]:
new_data = [{"Age": 42, "Income": "Low", "Student": "No", "Credit": "Excellent"}]
test_case = pd.DataFrame(new_data)

prediction = predict_sample(test_case.iloc[0], tree)
print(prediction)

Yes


# Q4 a - Bagging

In [35]:
def generate_bagging_dataset(X, y):
    """
    Creates a bagging or bootstrap sample of the same size as X,y
    """
    n = len(X)
    indices = np.random.randint(0, n, size=n)           # random indices with replacement
    X_bag = X.iloc[indices].reset_index(drop=True)
    y_bag = y.iloc[indices].reset_index(drop=True)

    sampled_set = set(indices)
    all_indices = set(range(n))
    oob_indices = np.array(list(all_indices - sampled_set))

    return X_bag, y_bag, oob_indices

def bagging_ensemble(X, y, n_trees=10, max_depth=5, min_samples_split=2):
    """
    Building an ensemble(here, 10) of decision trees via Bagging.
    Returns: trees and corresponding oob_lists
    """
    trees = []
    oob_lists = []

    for _ in range(n_trees):
        X_bag, y_bag, oob_indices = generate_bagging_dataset(X, y)

        tree = build_tree(X_bag, y_bag, depth=0, max_depth=max_depth,
                          min_samples_split=min_samples_split)

        trees.append(tree)
        oob_lists.append(oob_indices)

    return trees, oob_lists


In [36]:
def compute_oob_error(X, y, trees, oob_lists):
    n = len(X)
    correct_count = 0
    total_count = 0

    for i in range(n):
        # Finding trees that are OOB for sample i
        oob_tree_indices = []
        for t_idx, oob_idx_array in enumerate(oob_lists):
            if i in oob_idx_array:
                oob_tree_indices.append(t_idx)

        if len(oob_tree_indices) == 0:
            continue


        votes = []
        for t_idx in oob_tree_indices:
            tree = trees[t_idx]
            pred = predict_sample(X.iloc[i], tree)
            votes.append(pred)

        # Majority vote
        votes_series = pd.Series(votes)
        majority_pred = votes_series.value_counts().idxmax()

        if majority_pred == y.iloc[i]:
            correct_count += 1
        total_count += 1

    if total_count == 0:
        return None

    oob_error = 1 - (correct_count / total_count)
    return oob_error


## Training the Trees and Computing the OOB Error for Bagging
Upon trying multiple times, the best OOB Error for bagging was 0.375.

In [81]:
trees_bag, oob_lists_bag = bagging_ensemble(X, y, n_trees=10, max_depth=5, min_samples_split=2)

oob_error_bag = compute_oob_error(X, y, trees_bag, oob_lists_bag)
print("Bagging OOB error:", oob_error_bag)

Bagging OOB error: 0.375


# Q4 b - Random Forest

In [82]:
import random

# both functions are basically the same as the ones in Q3, only difference is that I am selecting a random subset of features to split
def find_best_split_random_features(X, y, max_features=2):
    """
    Modification from find_best_split: we use a random subset of features.
    """
    all_features = list(X.columns)
    if len(all_features) <= max_features:
        features_to_consider = all_features
    else:
        features_to_consider = random.sample(all_features, max_features)

    best_feature = None
    best_value = None
    best_gini_split = float('inf')
    best_left_index = None
    best_right_index = None

    n_samples = len(y)

    for feature in features_to_consider:
        unique_vals = X[feature].unique()

        if is_numeric(X[feature].dtype):
            sorted_vals = np.sort(unique_vals)
            for val in sorted_vals:
                left_index = X[feature] <= val
                right_index = X[feature] > val
                if left_index.sum() == 0 or right_index.sum() == 0:
                    continue

                left_gini = gini_impurity(y[left_index])
                right_gini = gini_impurity(y[right_index])
                w_left = left_index.sum() / n_samples
                w_right = right_index.sum() / n_samples
                gini_split = w_left * left_gini + w_right * right_gini

                if gini_split < best_gini_split:
                    best_gini_split = gini_split
                    best_feature = feature
                    best_value = val
                    best_left_index = left_index
                    best_right_index = right_index
        else:
            for val in unique_vals:
                left_index = X[feature] == val
                right_index = X[feature] != val
                if left_index.sum() == 0 or right_index.sum() == 0:
                    continue

                left_gini = gini_impurity(y[left_index])
                right_gini = gini_impurity(y[right_index])
                w_left = left_index.sum() / n_samples
                w_right = right_index.sum() / n_samples
                gini_split = w_left * left_gini + w_right * right_gini

                if gini_split < best_gini_split:
                    best_gini_split = gini_split
                    best_feature = feature
                    best_value = val
                    best_left_index = left_index
                    best_right_index = right_index

    return best_feature, best_value, best_gini_split, best_left_index, best_right_index


def build_tree_random_forest(X, y, depth=0, max_depth=5, min_samples_split=2, max_features=2):
    if len(np.unique(y)) == 1:
        return {"leaf": True, "prediction": y.iloc[0]}
    if depth >= max_depth or len(y) < min_samples_split:
        return {"leaf": True, "prediction": majority_class(y)}

    bf, bv, bg, left_idx, right_idx = find_best_split_random_features(X, y, max_features=max_features)
    if bf is None:
        return {"leaf": True, "prediction": majority_class(y)}

    node = {
        "leaf": False,
        "feature": bf,
        "value": bv,
        "gini": bg
    }

    X_left, y_left = X[left_idx], y[left_idx]
    X_right, y_right = X[right_idx], y[right_idx]

    node["left"] = build_tree_random_forest(X_left, y_left, depth+1, max_depth, min_samples_split, max_features)
    node["right"] = build_tree_random_forest(X_right, y_right, depth+1, max_depth, min_samples_split, max_features)

    return node


In [83]:
def random_forest_ensemble(X, y, n_trees=10, max_depth=5, min_samples_split=2, max_features=2):
    """
    Build a Random Forest
    """
    trees = []
    oob_lists = []

    for _ in range(n_trees):
        X_bag, y_bag, oob_indices = generate_bagging_dataset(X, y)

        tree = build_tree_random_forest(X_bag, y_bag, depth=0, max_depth=max_depth, min_samples_split=min_samples_split, max_features=max_features)

        trees.append(tree)
        oob_lists.append(oob_indices)

    return trees, oob_lists


## Training the Trees and Computing the OOB Error for Random Forest
Upon trying multiple times, the best OOB Error for RF was 0.25.

In [96]:
trees_rf, oob_lists_rf = random_forest_ensemble(X, y, n_trees=10, max_depth=5, min_samples_split=2, max_features=2)

oob_error_rf = compute_oob_error(X, y, trees_rf, oob_lists_rf)
print("Random Forest OOB error:", oob_error_rf)

Random Forest OOB error: 0.25
